# Question à choix multiples

Une tâche à choix multiples est similaire à la réponse à une question, sauf que plusieurs réponses candidates sont fournies avec un contexte et que le modèle est entraîné à sélectionner la bonne réponse.

Ce guide vous montrera comment :

1. Fine-tune [BERT](https://huggingface.co/bert-base-uncased) sur la configuration "régulière" de l'ensemble de données [SWAG](https://huggingface.co/datasets/swag) pour sélectionner la meilleure réponse à partir d'options multiples et d'un certain contexte.
2. Utilisez votre modèle fine-tune pour l'inférence.

<Tip>
La tâche illustrée dans ce tutoriel est prise en charge par les architectures de modèle suivantes :

<!--This tip is automatically generated by `make fix-copies`, do not fill manually!-->

[ALBERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/albert), [BERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/bert), [BigBird](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/big_bird), [CamemBERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/camembert), [CANINE](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/canine), [ConvBERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/convbert), [Data2VecText](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/data2vec-text), [DeBERTa-v2](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/deberta-v2), [DistilBERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/distilbert), [ELECTRA](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/electra), [ERNIE](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/ernie), [ErnieM](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/ernie_m), [FlauBERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/flaubert), [FNet](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/fnet), [Funnel Transformer](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/funnel), [I-BERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/ibert), [Longformer](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/longformer), [LUKE](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/luke), [MEGA](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mega), [Megatron-BERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/megatron-bert), [MobileBERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mobilebert), [MPNet](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mpnet), [Nezha](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/nezha), [Nyströmformer](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/nystromformer), [QDQBert](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/qdqbert), [RemBERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/rembert), [RoBERTa](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/roberta), [RoBERTa-PreLayerNorm](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/roberta-prelayernorm), [RoCBert](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/roc_bert), [RoFormer](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/roformer), [SqueezeBERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/squeezebert), [XLM](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/xlm), [XLM-RoBERTa](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/xlm-roberta), [XLM-RoBERTa-XL](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/xlm-roberta-xl), [XLNet](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/xlnet), [X-MOD](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/xmod), [YOSO](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/yoso)

<!--End of the generated tip-->

</Tip>

Avant de commencer, assurez-vous que toutes les bibliothèques nécessaires sont installées :

```bash
pip install transformers datasets evaluate
```



In [ ]:
!pip install datasets transformers

Vous devez ensuite installer Git-LFS. Décommentez les instructions suivantes :

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


Assurez-vous que votre version de Transformers est au moins 4.11.0 car la fonctionnalité a été introduite dans cette version :

In [ ]:
import transformers

print(transformers.__version__)

4.35.0


Nous téléchargeons également rapidement des données télémétriques, qui nous indiquent quels exemples et quelles versions de logiciels sont utilisés, afin que nous sachions où concentrer nos efforts de maintenance. Nous ne recueillons (ni ne nous intéressons à) aucune information personnelle identifiable, mais si vous préférez ne pas être compté, n'hésitez pas à sauter cette étape ou à supprimer entièrement cette cellule.

In [ ]:
from transformers.utils import send_example_telemetry

send_example_telemetry("multiple_choice_notebook", framework="pytorch")

# Fine-tuning   d'un modèle sur une tâche à choix multiples

Dans ce carnet, nous verrons comment affiner l'un des modèles [🤗 Transformers](https://github.com/huggingface/transformers) pour une tâche de choix multiple, qui est la tâche de sélectionner les entrées les plus plausibles dans une sélection donnée. L'ensemble de données utilisé ici est [SWAG](https://www.aclweb.org/anthology/D18-1009/) mais vous pouvez adapter le prétraitement à tout autre ensemble de données à choix multiples que vous souhaitez, ou à vos propres données. SWAG est un jeu de données sur le raisonnement de bon sens, où chaque exemple décrit une situation puis propose quatre options qui pourraient y répondre.

Ce bloc-notes est conçu pour fonctionner avec n'importe quel point de contrôle de modèle provenant du [Model Hub] (https://huggingface.co/models), à condition que ce modèle ait une version avec une tête à choix multiple. En fonction de votre modèle et du GPU que vous utilisez, il se peut que vous deviez ajuster la taille du lot pour éviter les erreurs de sortie de mémoire. Réglez ces deux paramètres, et le reste du carnet devrait fonctionner sans problème :

In [ ]:
model_checkpoint = "bert-base-uncased"
batch_size = 16

## Chargement du jeu de données

Nous utiliserons la bibliothèque [🤗 Datasets](https://github.com/huggingface/datasets) pour télécharger les données. Cela peut être facilement fait avec les fonctions `load_dataset`.

In [ ]:
from datasets import load_dataset, load_metric

`load_dataset` mettra en cache le jeu de données pour éviter de le télécharger à nouveau la prochaine fois que vous exécuterez cette cellule.

In [ ]:
datasets = load_dataset("swag", "regular")

L'objet `dataset` lui-même est [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), qui contient une clé pour l'ensemble d'apprentissage, de validation et de test (avec plus de clés pour l'ensemble de validation et de test non apparié dans le cas particulier de `mnli`).

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label'],
        num_rows: 73546
    })
    validation: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label'],
        num_rows: 20006
    })
    test: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label'],
        num_rows: 20005
    })
})

Pour accéder à un élément réel, vous devez d'abord sélectionner une division, puis donner un index :

In [ ]:
datasets["train"][0]

{'video-id': 'anetv_jkn6uvmqwh4',
 'fold-ind': '3416',
 'startphrase': 'Members of the procession walk down the street holding small horn brass instruments. A drum line',
 'sent1': 'Members of the procession walk down the street holding small horn brass instruments.',
 'sent2': 'A drum line',
 'gold-source': 'gold',
 'ending0': 'passes by walking down the street playing their instruments.',
 'ending1': 'has heard approaching them.',
 'ending2': "arrives and they're outside dancing and asleep.",
 'ending3': 'turns the lead singer watches the performance.',
 'label': 0}

Pour avoir une idée de l'aspect des données, la fonction suivante montre quelques exemples choisis au hasard dans l'ensemble de données.

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Il n'est pas possible de sélectionner plus d'éléments qu'il n'y en a dans l'ensemble de données.."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,video-id,fold-ind,startphrase,sent1,sent2,gold-source,ending0,ending1,ending2,ending3,label
0,lsmdc0025_THE_LORD_OF_THE_RINGS_THE_RETURN_OF_THE_KING-61837,4747,A little girl toddles up to greet him. Someone cotton,A little girl toddles up to greet him.,Someone cotton,gold,throws a horse onto the cart causing a branch to hit.,arabian is vertical and rugged.,- spreads it back on the turntable.,steps up and kisses someone on the cheek.,3
1,lsmdc3033_HUGO-14943,19103,"Someone kicks the inspector in the shin, causing him to lose his grip. Someone","Someone kicks the inspector in the shin, causing him to lose his grip.",Someone,gen,is dragged out of the water.,holds him international his wounded gaze.,"grins, just keeps his dildo retreating into the garage.",turns away from the door bolt.,0
2,anetv_6HyNydVIji4,2255,We see a lady performing a song on a flute. The girl,We see a lady performing a song on a flute.,The girl,gold,pauses and plays again.,spins around wearing several lights.,spins and smiles at the camera.,lifts the baby up in the air.,0
3,lsmdc3017_CHRONICLE-7167,14248,"With his free hand, he grabs the camera. Sharing his point - of - view, we","With his free hand, he grabs the camera.","Sharing his point - of - view, we",gen,see someone lying down from his attempt.,see someone giving his instructions.,glimpse his show collapse on the pavement.,see it change into rhythm.,1
4,lsmdc3019_COLOMBIANA-8317,11548,He returns to his seat. Someone,He returns to his seat.,Someone,gold,beams beside someone as he rubs her chest.,sits at his desk.,sips his drink as he watches his friend leave.,walks down the corridor.,2
5,lsmdc1008_Spider-Man2-76017,18668,Someone crashes into a set of shelves filled with spherical bombs. There,Someone crashes into a set of shelves filled with spherical bombs.,There,gold,takes possession of the surging metal.,someone and his friends follow each and open his mechanics for the last remark on the chess piece.,are row upon row of the weapons his father once used.,"someone reaches into the passenger window, moves the wall punctuated back from the cabinet side - first as kids rush out.",2
6,lsmdc3058_RUBY_SPARKS-28352,3237,The little dog runs up to her. Someone,The little dog runs up to her.,Someone,gold,appears from someone in the pantry.,takes off her bike helmet and shakes out her hair.,"flops back, then drops the rake, clutching his chest.",pulls her down with breaks.,1
7,lsmdc3034_IDES_OF_MARCH-3092,8246,"Leaning forward, she brings a hand to her mouth and shifts her hopeless, tearful gaze. Now someone","Leaning forward, she brings a hand to her mouth and shifts her hopeless, tearful gaze.",Now someone,gold,"sprints down the hospital dock and sets back onto the ridge, wearing the dress and carrying a suitcase.",wrenches the wheel to the cooling counter.,"marches purposefully down a sidewalk, his jaw set tight.","stares at the lad in the swimming costume, the lights show as him walking through the rain.",2
8,anetv_IgyBIt3GTAU,7205,"The man then takes a small can of wax and puts it on a piece of wood that he is holding in his hand. After dabbing the wax with a white rag, he",The man then takes a small can of wax and puts it on a piece of wood that he is holding in his hand.,"After dabbing the wax with a white rag, he",gold,"removes the garment, putting it on, and tastes it with it.",begins running his fingers over the wood and showing other pieces of wood that the product has been used on.,begins to make the shape closeups with a lot of wax.,shows the wax again wrapped in a parcel.,1
9,lsmdc1056_Rambo-97140,6662,They raise their rifles and someone leaps onto someone to shield her. Someone,They raise their rifles and someone leaps onto someone to shield her.,Someone,gold,gazes into her eyes.,"punches someone, knocking him to the floor.",ducks into the light.,shoots one of them.,3


Chaque exemple de l'ensemble de données a un contexte composé d'une première phrase (dans le champ `sent1`) et d'une introduction à la deuxième phrase (dans le champ `sent2`). Ensuite, quatre fins possibles sont données (dans les champs `ending0`, `ending1`, `ending2` et `ending3`) et le modèle doit choisir la bonne (indiquée dans le champ `label`). La fonction suivante nous permet de mieux visualiser un exemple donné :

In [ ]:
def show_one(example):
    print(f"Context: {example['sent1']}")
    print(f"  A - {example['sent2']} {example['ending0']}")
    print(f"  B - {example['sent2']} {example['ending1']}")
    print(f"  C - {example['sent2']} {example['ending2']}")
    print(f"  D - {example['sent2']} {example['ending3']}")
    print(f"\nGround truth: option {['A', 'B', 'C', 'D'][example['label']]}")

In [ ]:
show_one(datasets["train"][0])

Context: Members of the procession walk down the street holding small horn brass instruments.
  A - A drum line passes by walking down the street playing their instruments.
  B - A drum line has heard approaching them.
  C - A drum line arrives and they're outside dancing and asleep.
  D - A drum line turns the lead singer watches the performance.

Ground truth: option A


In [ ]:
show_one(datasets["train"][15])

Context: Now it's someone's turn to rain blades on his opponent.
  A - Someone pats his shoulder and spins wildly.
  B - Someone lunges forward through the window.
  C - Someone falls to the ground.
  D - Someone rolls up his fast run from the water and tosses in the sky.

Ground truth: option C


## Prétraitement des données

Avant de pouvoir introduire ces textes dans notre modèle, nous devons les prétraiter. Ceci est fait par un 🤗 Transformers `Tokenizer` qui va (comme son nom l'indique) tokeniser les entrées (y compris convertir les tokens en leurs IDs correspondants dans le vocabulaire pré-entraîné) et les mettre dans un format attendu par le modèle, ainsi que générer les autres entrées dont le modèle a besoin.

Pour faire tout cela, nous instancions notre tokenizer avec la méthode `AutoTokenizer.from_pretrained`, qui s'assurera que :

- nous obtenons un tokenizer qui correspond à l'architecture du modèle que nous voulons utiliser,
- nous téléchargeons le vocabulaire utilisé lors du pré-entraînement de ce point de contrôle spécifique.

Ce vocabulaire sera mis en cache, de sorte qu'il ne sera pas téléchargé à nouveau la prochaine fois que nous exécuterons la cellule.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Nous passons `use_fast=True` à l'appel ci-dessus pour utiliser un des tokenizers rapides (soutenu par Rust) de la bibliothèque 🤗 Tokenizers. Ces tokenizers rapides sont disponibles pour presque tous les modèles, mais si vous avez eu une erreur avec l'appel précédent, enlevez cet argument.

Vous pouvez appeler directement ce tokenizer sur une phrase ou une paire de phrases :

In [ ]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Selon le modèle que vous avez sélectionné, vous verrez différentes clés dans le dictionnaire retourné par la cellule ci-dessus. Elles n'ont pas beaucoup d'importance pour ce que nous faisons ici (sachez simplement qu'elles sont requises par le modèle que nous instancierons plus tard), vous pouvez en apprendre plus à leur sujet dans [ce tutoriel](https://huggingface.co/transformers/preprocessing.html) si cela vous intéresse.

Pour prétraiter notre jeu de données, nous aurons donc besoin des noms des colonnes contenant la/les phrase(s). Le dictionnaire suivant garde la trace de la tâche de correspondance avec les noms de colonnes :

Nous pouvons alors écrire la fonction qui va prétraiter nos échantillons. La partie la plus délicate est de mettre toutes les paires de phrases possibles dans deux grandes listes avant de les passer au tokenizer, puis de décompresser le résultat pour que chaque exemple ait quatre identifiants d'entrée, masques d'attention, etc.

Lors de l'appel au `tokenizer`, nous utilisons l'argument `truncation=True`. Cela garantit qu'une entrée plus longue que ce que le modèle sélectionné peut gérer sera tronquée à la longueur maximale acceptée par le modèle.

In [ ]:
ending_names = ["ending0", "ending1", "ending2", "ending3"]

def preprocess_function(examples):
    # Répétez chaque première phrase quatre fois pour aller avec les quatre possibilités de deuxième phrase.
    first_sentences = [[context] * 4 for context in examples["sent1"]]
    # Saisissez toutes les secondes phrases possibles pour chaque contexte.
    question_headers = examples["sent2"]
    second_sentences = [[f"{header} {examples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)]

    # Aplatir tout
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    # Tokenize
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    # Un-flatten
    return {k: [v[i:i+4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

Cette fonction fonctionne avec un ou plusieurs exemples. Dans le cas de plusieurs exemples, le tokenizer retournera une liste de listes de listes pour chaque clé : une liste de tous les exemples (ici 5), puis une liste de tous les choix (4) et une liste d'ID d'entrée (longueur variable ici puisque nous n'avons pas appliqué de padding) :

In [ ]:
examples = datasets["train"][:5]
features = preprocess_function(examples)
print(len(features["input_ids"]), len(features["input_ids"][0]), [len(x) for x in features["input_ids"][0]])

5 4 [30, 25, 30, 28]


Pour vérifier que nous n'avons rien fait en regroupant toutes les possibilités puis en les aplatissant, examinons les entrées décodées pour un exemple donné :

In [ ]:
idx = 3
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(4)]

['[CLS] a drum line passes by walking down the street playing their instruments. [SEP] members of the procession are playing ping pong and celebrating one left each in quick. [SEP]',
 '[CLS] a drum line passes by walking down the street playing their instruments. [SEP] members of the procession wait slowly towards the cadets. [SEP]',
 '[CLS] a drum line passes by walking down the street playing their instruments. [SEP] members of the procession makes a square call and ends by jumping down into snowy streets where fans begin to take their positions. [SEP]',
 '[CLS] a drum line passes by walking down the street playing their instruments. [SEP] members of the procession play and go back and forth hitting the drums while the audience claps for them. [SEP]']

Nous pouvons la comparer à la vérité terrain :

In [ ]:
show_one(datasets["train"][3])

Context: A drum line passes by walking down the street playing their instruments.
  A - Members of the procession are playing ping pong and celebrating one left each in quick.
  B - Members of the procession wait slowly towards the cadets.
  C - Members of the procession makes a square call and ends by jumping down into snowy streets where fans begin to take their positions.
  D - Members of the procession play and go back and forth hitting the drums while the audience claps for them.

Ground truth: option D


Cela semble correct, donc nous pouvons appliquer cette fonction sur tous les exemples de notre jeu de données, nous utilisons simplement la méthode `map` de notre objet `dataset` que nous avons créé plus tôt. Cela appliquera la fonction sur tous les éléments de tous les splits de `dataset`, ainsi nos données d'entraînement, de validation et de test seront prétraitées en une seule commande.

In [ ]:
encoded_datasets = datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/73546 [00:00<?, ? examples/s]

Map:   0%|          | 0/20006 [00:00<?, ? examples/s]

Map:   0%|          | 0/20005 [00:00<?, ? examples/s]

Mieux encore, les résultats sont automatiquement mis en cache par la bibliothèque 🤗 Datasets pour éviter de perdre du temps à cette étape la prochaine fois que vous exécuterez votre notebook. La bibliothèque 🤗 Datasets est normalement assez intelligente pour détecter quand la fonction que vous passez à map a changé (et nécessite donc de ne pas utiliser les données du cache). Par exemple, elle détectera correctement si vous changez la tâche dans la première cellule et réexécutez le carnet. 🤗 Datasets vous avertit lorsqu'il utilise des fichiers en cache, vous pouvez passer `load_from_cache_file=False` dans l'appel à `map` pour ne pas utiliser les fichiers en cache et forcer le prétraitement à être appliqué à nouveau.

Notez que nous avons passé `batched=True` pour encoder les textes par lots ensemble. Cela permet de tirer le meilleur parti du tokenizer rapide que nous avons chargé plus tôt, qui utilisera le multithreading pour traiter les textes d'un lot simultanément.

## Affiner le modèle

Maintenant que nos données sont prêtes, nous pouvons télécharger le modèle pré-entraîné et l'ajuster. Puisque notre tâche concerne les choix multiples, nous utilisons la classe `AutoModelForMultipleChoice`. Comme pour le tokenizer, la méthode `from_pretrained` téléchargera et mettra en cache le modèle pour nous.

In [ ]:
model_checkpoint

'bert-base-uncased'

In [ ]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint)

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


L'avertissement nous dit que nous jetons certains poids (les couches `vocab_transform` et `vocab_layer_norm`) et que nous en initialisons d'autres au hasard (les couches `pre_classifier` et `classifier`). C'est tout à fait normal dans ce cas, car nous supprimons la tête utilisée pour pré-entraîner le modèle sur un objectif de modélisation de langage masqué et nous la remplaçons par une nouvelle tête pour laquelle nous n'avons pas de poids pré-entraînés, donc la bibliothèque nous avertit que nous devrions affiner ce modèle avant de l'utiliser pour l'inférence, ce qui est exactement ce que nous allons faire.

Pour instancier un `Trainer`, nous aurons besoin de définir trois autres choses. Le plus important est le [`TrainingArguments`] (https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), qui est une classe qui contient tous les attributs pour personnaliser l'entraînement. Elle requiert un nom de dossier, qui sera utilisé pour sauvegarder les points de contrôle du modèle, et tous les autres arguments sont optionnels :

In [ ]:
#pip install accelerate -U

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-swag",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    #push_to_hub=True,
)

Ici, nous fixons l'évaluation à la fin de chaque époch, nous ajustons le taux d'apprentissage, nous utilisons le `batch_size` défini en haut du notebook et nous personnalisons le nombre d'épochs pour l'entraînement, ainsi que la décroissance des poids.

Le dernier argument permet de tout configurer pour que nous puissions pousser le modèle vers le [Hub](https://huggingface.co/models) régulièrement pendant l'apprentissage. Enlevez-le si vous n'avez pas suivi les étapes d'installation en haut du notebook. Si vous voulez sauvegarder votre modèle localement sous un nom différent de celui du dépôt dans lequel il sera poussé, ou si vous voulez pousser votre modèle sous une organisation et non sous votre espace de noms, utilisez l'argument `hub_model_id` pour définir le nom du dépôt (il doit s'agir du nom complet, y compris votre espace de noms : par exemple `"sgugger/bert-finetuned-swag"` ou `"huggingface/bert-finetuned-swag"`).

Ensuite, nous devons dire à notre `Trainer` comment former des lots à partir des entrées prétraitées. Nous n'avons pas encore fait de padding car nous allons padder chaque lot à la longueur maximale à l'intérieur du lot (au lieu de le faire avec la longueur maximale de l'ensemble des données). C'est le travail du *collateur de données*. Un collecteur de données prend une liste d'exemples et les convertit en un lot (en appliquant, dans notre cas, un remplissage). Comme il n'y a pas de collateur de données dans la bibliothèque qui fonctionne pour notre problème spécifique, nous allons en écrire un, adapté du `DataCollatorWithPadding` :

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

Lorsqu'il est appelé sur une liste d'exemples, il aplatit toutes les entrées/masques d'attention, etc. dans de grandes listes qu'il passe à la méthode `tokenizer.pad`. Cela retournera un dictionnaire avec de grands tenseurs (de la forme `(batch_size * 4) x seq_length`) que nous décompresserons ensuite.

Nous pouvons vérifier que ce collateur de données fonctionne sur une liste de features, nous devons juste nous assurer de supprimer toutes les features qui ne sont pas des inputs acceptés par notre modèle (ce que le `Trainer` fera automatiquement pour nous par la suite) :

In [ ]:
accepted_keys = ["input_ids", "attention_mask", "label"]
features = [{k: v for k, v in encoded_datasets["train"][i].items() if k in accepted_keys} for i in range(10)]
batch = DataCollatorForMultipleChoice(tokenizer)(features)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.




```
# Ce texte est au format code
```

Encore une fois, tous ces aplatissements/déplatissements sont des sources d'erreurs potentielles :

In [ ]:
[tokenizer.decode(batch["input_ids"][8][i].tolist()) for i in range(4)]

['[CLS] someone walks over to the radio. [SEP] someone hands her another phone. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] someone walks over to the radio. [SEP] someone takes the drink, then holds it. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] someone walks over to the radio. [SEP] someone looks off then looks at someone. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] someone walks over to the radio. [SEP] someone stares blearily down at the floor. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]']

In [ ]:
show_one(datasets["train"][8])

Context: Someone walks over to the radio.
  A - Someone hands her another phone.
  B - Someone takes the drink, then holds it.
  C - Someone looks off then looks at someone.
  D - Someone stares blearily down at the floor.

Ground truth: option D


Tout va bien !

La dernière chose à définir pour notre `Trainer` est comment calculer les métriques à partir des prédictions. Nous devons définir une fonction pour cela, qui utilisera simplement la `métrie` que nous avons chargée plus tôt, le seul prétraitement que nous avons à faire est de prendre l'argmax de nos logits prédits :

In [ ]:
import numpy as np

def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

Il ne nous reste plus qu'à passer tout cela avec nos jeux de données au `Trainer` :

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer),
    compute_metrics=compute_metrics,
)

Nous pouvons maintenant affiner notre modèle en appelant simplement la méthode `train` :

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-31-3435b262f1ae>", line 1, in <cell line: 1>
    trainer.train()
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1555, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1860, in _inner_training_loop
    tr_loss_step = self.training_step(model, inputs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2734, in training_step
    self.accelerator.backward(loss)
  File "/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py", line 1989, in backward
    loss.backward(**kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/_tensor.py", line 492, in backward
    torch.autograd.backward(
  File "/usr/local/lib/python3.10/dist-package

TypeError: ignored

## Inférence

Maintenant que vous avez mis au point un modèle, vous pouvez l'utiliser pour l'inférence !

Trouvez un texte et deux réponses possibles :

In [ ]:
show_one(datasets["validation"][0])

Context: Students lower their eyes nervously.
  A - She pats her shoulder, then saunters toward someone.
  B - She turns with two students.
  C - She walks slowly towards someone.
  D - She wheels around as her dog thunders out.

Ground truth: option C


In [ ]:
prompt = "Students lower their eyes nervously."
candidate1 = "She pats her shoulder, then saunters toward someone."
candidate2 = "She turns with two students."
candidate3="She walks slowly towards someone."
candidate4="She wheels around as her dog thunders out."

Tokeniser chaque paire d'invite et de réponse de candidat et renvoyer des tenseurs PyTorch. Vous devez également créer des `labels` :

In [ ]:
import torch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("ncduy/bert-base-uncased-finetuned-swag")
inputs = tokenizer([[prompt, candidate1], [prompt, candidate2],[prompt, candidate3],[prompt, candidate4]], return_tensors="pt", padding=True)
labels = torch.tensor(2).unsqueeze(0)

Passez vos entrées et vos étiquettes au modèle et renvoyez les `logits` :

In [ ]:
from transformers import AutoModelForMultipleChoice

model = AutoModelForMultipleChoice.from_pretrained("ncduy/bert-base-uncased-finetuned-swag")
outputs = model(**{k: v.unsqueeze(0) for k, v in inputs.items()}, labels=labels)
logits = outputs.logits

Obtenez la classe avec la probabilité la plus élevée :

In [ ]:
predicted_class = logits.argmax().item()
predicted_class
print('Predicted: option %s'%['A','B','C','D'][predicted_class])

Predicted: option C
